In [3]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
/content/drive/MyDrive/Brain Tumor.zip



In [5]:
# Mount Google Drive
from google.colab import drive
import os
import sys
import pandas as pd
import zipfile
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adamax

# Mount Google Drive
drive.mount('/content/drive')

drive_path = '/content/drive/MyDrive/Brain Tumor.zip'
zip_path = drive_path
extract_path = '/content/brain_tumor_data'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

train_data_dir = os.path.join(extract_path, 'Training')
test_data_dir = os.path.join(extract_path, 'Testing')

filepaths, labels = [], []

def collect_data(data_dir):
    for class_folder in os.listdir(data_dir):
        class_path = os.path.join(data_dir, class_folder)
        if os.path.isdir(class_path):
            for file in os.listdir(class_path):
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    filepaths.append(os.path.join(class_path, file))
                    labels.append(class_folder)

collect_data(train_data_dir)
collect_data(test_data_dir)

data_df = pd.DataFrame({'filepaths': filepaths, 'labels': labels})
train_df, temp_df = train_test_split(data_df, train_size=0.7, shuffle=True, random_state=42)
valid_df, test_df = train_test_split(temp_df, train_size=0.5, shuffle=True, random_state=42)

batch_size = 16
img_size = (224, 224)
tr_gen = ImageDataGenerator(rescale=1./255)
ts_gen = ImageDataGenerator(rescale=1./255)

train_gen = tr_gen.flow_from_dataframe(train_df, x_col='filepaths', y_col='labels', target_size=img_size, class_mode='categorical', color_mode='rgb', shuffle=True, batch_size=batch_size)
valid_gen = ts_gen.flow_from_dataframe(valid_df, x_col='filepaths', y_col='labels', target_size=img_size, class_mode='categorical', color_mode='rgb', shuffle=True, batch_size=batch_size)
test_gen = ts_gen.flow_from_dataframe(test_df, x_col='filepaths', y_col='labels', target_size=img_size, class_mode='categorical', color_mode='rgb', shuffle=False, batch_size=batch_size)

img_shape = (224, 224, 3)
class_count = len(train_gen.class_indices.keys())

model = Sequential([
    Flatten(input_shape=img_shape),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(class_count, activation='softmax')
])

model.compile(optimizer=Adamax(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 10
history = model.fit(train_gen, epochs=epochs, validation_data=valid_gen, shuffle=False)

model_path = "/content/drive/MyDrive/Brain Tumor Detection/mlp_model.h5"
model.save(model_path)
print(f"Model saved at {model_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 4916 validated image filenames belonging to 4 classes.
Found 1053 validated image filenames belonging to 4 classes.
Found 1054 validated image filenames belonging to 4 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
308/308 ━━━━━━━━━━━━━━━━━━━━ 163s 525ms/step - accuracy: 0.5507 - loss: 2.9561 - val_accuracy: 0.7436 - val_loss: 0.7280
Epoch 2/10
308/308 ━━━━━━━━━━━━━━━━━━━━ 154s 499ms/step - accuracy: 0.7733 - loss: 0.6654 - val_accuracy: 0.7740 - val_loss: 0.7155
Epoch 3/10
308/308 ━━━━━━━━━━━━━━━━━━━━ 196s 480ms/step - accuracy: 0.7977 - loss: 0.5534 - val_accuracy: 0.8101 - val_loss: 0.5850
Epoch 4/10
308/308 ━━━━━━━━━━━━━━━━━━━━ 161s 523ms/step - accuracy: 0.8332 - loss: 0.4661 - val_accuracy: 0.7417 - val_loss: 0.7871
Epoch 5/10
308/308 ━━━━━━━━━━━━━━━━━━━━ 149s 485ms/step - accuracy: 0.8232 - loss: 0.4754 - val_accuracy: 0.8234 - val_loss: 0.5636
Epoch 6/10
308/308 ━━━━━━━━━━━━━━━━━━━━ 153s 497ms/step - accuracy: 0.8846 - loss: 0.3022 - val_accuracy: 0.7721 - val_loss: 0.7182
Epoch 7/10
308/308 ━━━━━━━━━━━━━━━━━━━━ 155s 501ms/step - accuracy: 0.8899 - loss: 0.2978 - val_accuracy: 0.7673 - val_loss: 0.7880
Epoch 8/10
308/308 ━━━━━━━━━━━━━━━━━━━━ 166s 541ms/step - accuracy: 0.8729 -

Model saved at /content/drive/MyDrive/Brain Tumor Detection/mlp_model.h5
